In [8]:
import pandas as pd
import numpy as np

In [9]:
df = pd.read_csv("../data/TIGPSw1_s.csv")

C:\Users\as123\AppData\Local\Temp\ipykernel_33228\2047782468.py:1: DtypeWarning: Columns (19,29,152,324,340,368,383,481,483,505,507,529,531,553,555,577,579,603,627) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/TIGPSw1_s.csv")


In [16]:
# -*- coding: utf-8 -*-
# 人工建立的變項 ID 到變項說明映射字典 (id_map)
# 請根據 TIGPSw1_cb_s.pdf Codebook 文件手動補充與核對

id_map = {
    # --- Metadata / ID Variables ---
    "qb_code": "題本編號",
    "q_name": "問卷名稱",
    "nstudent_id": "學生 ID", # Codebook 寫 nstudent id，您提供的 CSV 標頭是 nstudent_id
    "nschool_id": "學校 ID", # Codebook 寫 nschool id，您提供的 CSV 標頭是 nschool_id
    "nclass": "班級 (區別同一學校內之受測班級)",
    "school_class": "學校班級 ID (由學校ID與校內班級編碼組成)",
    "school_type": "學校類型",
    "seat_n": "座號",

    # --- (一) 個人與家庭基本資料 ---
    "as1": "s1. 請問你的性別(生理性別)?", # 在說明前加上題號 sX. 方便對照
    "as2": "s2. 請問你是哪一年出生?", # (數值，無需值標籤轉換，但可加說明)
    "as3": "s3. 請問你親生父母的婚姻狀態?",
    "aks3": "s3. 請問你親生父母的婚姻狀態?(11)其他，請說明", # 對應的文字欄位
    "as4": "s4. 請問你的主要照顧者（家長）的一些情形",
    "as4a1": "s4a(一). 請問你的第一位主要照顧者是你的?",
    "aks4a1": "s4a(一). 請問你的第一位主要照顧者是你的?(07)其他，請說明",
    "as4b1": "s4b(一). 請問你的第一位主要照顧者 民國出生年?",
    "as4c1": "s4c(一). 請問你的第一位主要照顧者是哪裡人?",
    "aks4c1": "s4c(一). 請問你的第一位主要照顧者是哪裡人?(08)其他，請說明",
    "as4c1_4": "s4c(一). A 請問你的第一位主要照顧者是哪一族?", # 原住民細項
    "as4c1_7": "s4c(一). B 請問你的第一位主要照顧者是哪一國?", # 其他國籍細項
    "aks4c1_7": "s4c(一). B 請問你的第一位主要照顧者是哪一國?(13)其他，請說明",
    "as4d1": "s4d(一). 請問你的第一位主要照顧者的最高學歷?",
    "as4e1": "s4e(一). 請問你的第一位主要照顧者的健康情形?",
    "as4a2": "s4a(二). 請問你的第二位主要照顧者是你的?", # 第二照顧者開始
    "aks4a2": "s4a(二). 請問你的第二位主要照顧者是你的?(07)其他,請說明",
    "as4b2": "s4b(二). 請問你的第二位主要照顧者 民國出生年?",
    "as4c2": "s4c(二). 請問你的第二位主要照顧者是哪裡人?",
    "aks4c2": "s4c(二). 請問你的第二位主要照顧者是哪裡人？(08)其他，請說明",
    "as4c2_4": "s4c(二). 請問你的第二位主要照顧者是哪一族？",
    "as4c2_7": "s4c(二). 請問你的第二位主要照顧者是哪一國？",
    "aks4c2_7": "s4c(二). 請問你的第二位主要照顧者是哪一國？(13)其他，請說明",
    "as4d2": "s4d(二). 請問你的第二位主要照顧者的最高學歷?",
    "as4e2": "s4e(二). 請問你的第二位主要照顧者的健康情形?",
    "as5": "s5. 請問你是否有兄弟姐妹?",
    "as6": "s6. 那你有幾個兄弟姐妹",
    "as6a": "s6a. 請問你的兄弟姊妹的人數: 兄 __ 人", # Codebook 寫 asba，您提供的 CSV 是 as6a
    "as6b": "s6b. 請問你的兄弟姊妹的人數: 姐 __ 人",
    "as6c": "s6c. 請問你的兄弟姊妹的人數: 弟 __ 人",
    "as6d": "s6d. 請問你的兄弟姊妹的人數: 妹 __ 人",
    "as7": "s7. 以下描述符不符合你與兄弟姊妹之間的相處？(B卷)",
    "as7a": "s7a. 我與兄弟姊妹之間相處融洽,會相互照顧與分享。(B卷)", # B卷作答
    "as7b": "s7b. 我與兄弟姊妹之間常有衝突。(B卷)",
    "as7c": "s7c. 我與兄弟姊妹之間會彼此互相比較。(B卷)",
    "as8": "s8. 你目前與下列哪些人同住？",
    "as8_1": "s8. 親生父親", # 多選題拆分
    "as8_2": "s8. 親生母親",
    "as8_3": "s8. 養父/繼父",
    "as8_4": "s8. 養母/繼母",
    "as8_5": "s8. 已婚兄弟",
    "as8_6": "s8. 已婚姊妹",
    "as8_7": "s8. 未婚兄弟",
    "as8_8": "s8. 未婚姊妹",
    "as8_9": "s8. 兄弟的配偶",
    "as8_10": "s8. 姊妹的配偶",
    "as8_11": "s8. 祖父",
    "as8_12": "s8. 祖母",
    "as8_13": "s8. 外祖父",
    "as8_14": "s8. 外祖母",
    "as8_15": "s8. 伯父、伯母",
    "as8_16": "s8. 叔叔、嬸嬸",
    "as8_17": "s8. 姑姑、姑丈",
    "as8_18": "s8. 舅舅、舅媽",
    "as8_19": "s8. 阿姨、姨丈",
    "as8_20": "s8. 表／堂兄姊",
    "as8_21": "s8. 表／堂弟妹",
    "as8_22": "s8. 看護或幫傭",
    "as8_23": "s8. 其他",
    "aks8_23": "s8. 其他，（請說明）",
    "as9": "s9. 下列關於你家中生活的情形，你認為符不符合？",
    "as9a": "s9a. 作決定時,家人會彼此商量。", # Likert scale
    "as9b": "s9b. 家人喜歡共度休閒時光。",
    "as9c": "s9c. 當有家庭活動時,我們家每個人都會參加。",
    "as9d": "s9d. 家人會接納彼此的朋友。",
    "as9e": "s9e. 我受到挫折時,總可以從我的家人那裡得到安慰。",
    "as9f": "s9f. 當我需要幫忙或忠告時,我可以依賴我的家人。",
    "as10": "s10. 請問你認為家裡的經濟狀況為何?",
    "as11": "s11. 請問您目前有沒有宗教信仰?",
    "aks11_7": "s11. 請問教派是? (基督教)", # 稍微整理說明
    "aks11_9": "s11. 請問您目前有沒有宗教信仰?(09)其他,請說明",
    "as12": "s12. 過去一年，你家長和你相處情形如何？(B卷)",
    "as12a": "s12a. 過去一年,家長和你相處情形: 批評你或指責你。(B卷)",
    "as12b": "s12b. 過去一年,家長和你相處情形: 理解你、肯定你的想法。(B卷)",
    "as12c": "s12c. 過去一年,家長和你相處情形: 關心你。(B卷)",
    "as12d": "s12d. 過去一年,家長和你相處情形: 提供解決問題的經驗，給你參考。(B卷)",
    "as12e": "s12e. 過去一年,家長和你相處情形: 對你不聞不問。(B卷)",
    "as12f": "s12f. 過去一年,家長和你相處情形: 跟你常有衝突，彼此關係緊張。(B卷)",
    "as12g": "s12g. 過去一年,家長和你相處情形: 知道你每天的行蹤。(B卷)",
    "as12h": "s12h. 過去一年,家長和你相處情形: 同樣一件事，他們有時處罰小孩，有時不處罰。(B卷)",
    "as12i": "s12i. 過去一年,家長和你相處情形: 處罰你時會打你。(B卷)",
    "as12j": "s12j. 過去一年,家長和你相處情形: 有關你的事，會告訴你他們的想法。(B卷)",
    # --- (二) 學校生活與學業 ---
    "as13": "s13. 請問你對你目前的學校生活滿意嗎?",
    "as14": "s14. 下列是有關學校的描述，你同不同意？",
    "as14a": "s14a. 我喜歡學校。",
    "as14b": "s14b. 我不認為我是學校的一份子。",
    "as14c": "s14c. 我以我的學校為榮。",
    "as15": "s15. 以下對你學校的描述你覺得符不符合？",
    "as15a": "s15a. 學生逃學或翹課情況嚴重。",
    "as15b": "s15b. 學生抽菸或喝酒問題嚴重。",
    "as16": "s16. 下列有關你們班上的說法，你認為符不符合目前的狀況？",
    "as16a": "s16a. 當我需要他們的時候，我的同學們總是會適時伸出援手。",
    "as16b": "s16b. 我不喜歡與班上同學來往。",
    "as16c": "s16c. 當班上的同學需要我的時候，我總是會適時伸出援手。",
    "as16d": "s16d. 我們班的同學相親相愛，猶如一家人。",
    "as16e": "s16e. 與其他班級比較，我們班的同學彼此之間，最互相信賴。",
    "as17": "s17. 請問你的座號是？___號 (A卷)",
    "as18": "s18. 以下要了解你與班上同學在網路上以及實際生活中的相處狀況，請選出你最喜歡與最不喜歡的同學座號，最多可以選擇五位。 (A卷)",
    "as18a": "s18a. 在網路上，你喜歡跟班上哪些同學互動（例如一起打遊戲、講電話、聊天等）？",
    "as18a1": "s18a1. (1)",
    "as18a2": "s18a2. (2)",
    "as18a3": "s18a3. (3)",
    "as18a4": "s18a4. (4)",
    "as18a5": "s18a5. (5)",
    "as18b": "s18b. 在網路上，你不喜歡跟班上哪些同學互動？",
    "as18b1": "s18b1. (1)",
    "as18b2": "s18b2. (2)",
    "as18b3": "s18b3. (3)",
    "as18b4": "s18b4. (4)",
    "as18b5": "s18b5. (5)",
    "as18c": "s18c. 在實際生活（非網路）中，你喜歡跟班上哪些同學互動（例如分組同組、一起吃飯、討論功課、聊心事等）？",
    "as18c1": "s18c1. (1)",
    "as18c2": "s18c2. (2)",
    "as18c3": "s18c3. (3)",
    "as18c4": "s18c4. (4)",
    "as18c5": "s18c5. (5)",
    "as18d": "s18d. 在實際生活（非網路）中，你不喜歡跟班上哪位同學互動？",
    "as18d1": "s18b1. (1)",
    "as18d2": "s18b2. (2)",
    "as18d3": "s18b3. (3)",
    "as18d4": "s18b4. (4)",
    "as18d5": "s18b5. (5)",
    "as19": "s19. 你跟得上學校課業進度嗎？",
    "as20": "s20. 你上學期的平均成績大約如何？",
    "as21": "s21. 家長對你的學業成績有些甚麼要求？",
    "aks21": "s21. 家長對你的學業成績有些甚麼要求？(05)其他，請說明",
    "as22": "s22. 上七年級以來，你有沒有參加校外補習（包含家教與家教班）？",
    "as22a": "s22a. 你補哪幾科？",
    "as22a1": "s22a1. 數學",
    "as22a2": "s22a2. 英語",
    "as22a3": "s22a3. 生物",
    "as22a4": "s22a4. 國文",
    "as22a5": "s22a5. 社會（史、地、公）",
    "as22a6": "s22a6. 資訊/電腦",
    "as22a7": "s22a7. 其他",
    "aks22a7": "s22a7. 其他（請說明）",
    "as23": "s23. 請問父母（家長）對你下列未來發展的期望程度如何？請從1 到10 分點選期望程度。",
    "as23a": "s23a. 他們期望我的學業成就可以令他們引以自豪(B卷)",
    "as23b": "s23b. 他們期望我將來可以有一份高薪的工作(B卷)",
    "as23c": "s23c. 他們期望我將來可以分擔家庭的經濟(B卷)",
    "as23d": "s23d. 他們期望我可以光宗耀祖(B卷)",
    "as24": "s24. 請問以下敘述是否符合你家長的情形？(B卷)",
    "as24a": "s24a. 家長知道我在學校主要的授課老師是誰。(B卷)",
    "as24b": "s24b. 家長知道我在學校的學習狀況（例如: 是否跟得上進度？）。(B卷)",
    "as24c": "s24c. 家長知道我在學校的課業內容（例如:現在教到哪裡？）。(B卷)",
    "as24d": "s24d. 家長會參加學校的家長會/學校日。(B卷)",
    "as24e": "s24e. 家長會為班上的活動或設備出錢出力。(B卷)",
    "as24f": "s24f. 家長會為班上的活動或設備出錢出力。(B卷)",
    "as24g": "s24g. 家長會花時間在我的學校功課上。(B卷)",
    "as24h": "s24h. 家長會幫我看聯絡簿。(B卷)",
    "as24i": "s24i. 家長會幫我買課本以外的評量、測驗卷等補充教材。(B卷)",
    "as24j": "s24j. 家長會上網幫我搜尋相關的教學材料或教學平台。(B卷)",
    "as25": "s25. 請問你們班有沒有下列群組（LINE 或臉書等群組）？(B卷)",
    "as25a": "s25a. 班上同學的群組，不包含導師。(B卷)",
    "as25b": "s25b. 班上同學的群組，導師有加入。(B卷)",
    "as26": "s26. 上七年級以來，你的導師是否曾主動聯絡你的家長？(B卷)",
    "as26a": "s26a. 導師多常聯絡你的家長？(B卷)",
    "as26b": "s26b. 導師通常是連絡哪位家長？(B卷)",
    "aks26b": "s26b. 導師通常是連絡哪位家長？(5)其他家長（請說明）(B卷)",
    "as27": "s27. 上七年級以來，你的家長是否曾主動聯絡導師？(B卷)",
    "as27a": "s27a.他們多常聯絡你的導師？(B卷)",
    "as27b": "s27b. 通常是誰連絡你的導師？(B卷)",
    "aks27b": "s27b. 通常是誰連絡你的導師？(5)其他家長（請說明）(B卷)",
    # --- (三) 網路生活經驗 ---
    "as28": "s28. 關於上網，你父親和你有沒有以下的情形？(B卷)",
    "as28a": "s28a. 會規定你什麼時候可以上網，什麼時候不行。(B卷)",
    "as28b": "s28b. 會禁止你上某些網站。(B卷)",
    "as28c": "s28c. 會限制你上網時間的長短。(B卷)",
    "as28d": "s28d. 會和你一起聊網路上發生的趣聞或事件。(B卷)",
    "as28e": "s28e. 會和你一起上網玩遊戲或追劇。(B卷)",
    "as28f": "s28f. 會陪你一起上網查資料或做作業。(B卷)",
    "as28g": "s28g. 會鼓勵你上網。(B卷)",
    "as28h": "s28h. 會和你討論使用網路的情形和心得。(B卷)",
    "as28i": "s28i. 認為擅長使用網路對你的未來很有幫助。(B卷)",
    "as28j": "s28j. 擔心你上網會產生健康問題（如：眼睛、體能）。(B卷)",
    "as28k": "s28k. 會擔心你面臨的網路風險（如：負面訊息、色情、霸凌、詐騙等）。 (B卷)",
    "as28l": "s28l. 擔心你的課業成績會因為上網而退步。(B卷)",
    "as28m": "s28m. 會問你在網上做什麼。(B卷)",
    "as28n": "s28n. 知道你在網路世界裡的朋友是誰。(B卷)",
    "as28o": "s28o. 會查看你的網頁瀏覽記錄。(B卷)",
    "as28p": "s28p. 會在你上網時碎碎念。(B卷)",
    "as28q": "s28q. 會因為你上網而責罵你。(B卷)",
    "as28r": "s28r. 會因為你在網路購物而和你起衝突。(B卷)",
    "as29": "s29. 關於上網，你母親和你有沒有以下的情形？(B卷)",
    "as29a": "s29a. 會規定你什麼時候可以上網，什麼時候不行。(B卷)",
    "as29b": "s29b. 會禁止你上某些網站。(B卷)",
    "as29c": "s29c. 會限制你上網時間的長短。(B卷)",
    "as29d": "s29d. 會和你一起聊網路上發生的趣聞或事件。(B卷)",
    "as29e": "s29e. 會和你一起上網玩遊戲或追劇。(B卷)",
    "as29f": "s29f. 會陪你一起上網查資料或做作業。(B卷)",
    "as29g": "s29g. 會鼓勵你上網。(B卷)",
    "as29h": "s29h. 會和你討論使用網路的情形和心得。(B卷)",
    "as29i": "s29i. 認為擅長使用網路對你的未來很有幫助。(B卷)",
    "as29j": "s29j. 擔心你上網會產生健康問題（如：眼睛、體能）。(B卷)",
    "as29k": "s29k. 會擔心你面臨的網路風險（如：負面訊息、色情、霸凌、詐騙等）。 (B卷)",
    "as29l": "s29l. 擔心你的課業成績會因為上網而退步。(B卷)",
    "as29m": "s29m. 會問你在網上做什麼。(B卷)",
    "as29n": "s29n. 知道你在網路世界裡的朋友是誰。(B卷)",
    "as29o": "s29o. 會查看你的網頁瀏覽記錄。(B卷)",
    "as29p": "s29p. 會在你上網時碎碎念。(B卷)",
    "as29q": "s29q. 會因為你上網而責罵你。(B卷)",
    "as29r": "s29r. 會因為你在網路購物而和你起衝突。(B卷)",
    "as30": "s30. 關於你上網，主要是誰在管？(B卷)",
    "aks30": "s30. 關於你上網，主要是誰在管？(9)其他人（請說明）(B卷)",
    "as31": "s31. 你有沒有社群媒體帳號？(B卷)",
    "as31a1": "s31a. 有哪些社群媒體的帳號？: LINE (B卷)",
    "as31a2": "s31a. 有哪些社群媒體的帳號？: Facebook (B卷)",
    "as31a3": "s31a. 有哪些社群媒體的帳號？: Instagram (B卷)",
    "as31a4": "s31a. 有哪些社群媒體的帳號？: WeChat  (B卷)",
    "as31a5": "s31a. 有哪些社群媒體的帳號？: Dcard (B卷)",
    "as31a6": "s31a. 有哪些社群媒體的帳號？: 抖音（Tiktok） (B卷)",
    "as31a7": "s31a. 有哪些社群媒體的帳號？: 小紅書 (B卷)",
    "as31a8": "s31a. 有哪些社群媒體的帳號？: 批踢踢（PTT） (B卷)",
    "as31a9": "s31a. 有哪些社群媒體的帳號？: Discord (B卷)",
    "as31a10": "s31a. 有哪些社群媒體的帳號？: 其他（請說明） (B卷)",
    "aks31a10": "s31a. 有哪些社群媒體的帳號？(10)其他（請說明）: 其他（請說明） (B卷)",
    "as32": "s32. 請問你和家長（父母）的即時通訊情形：（父母合併為：家長（父母））",
    "as32a": "s32a. 你平常和他們使用即時通訊的頻率如何？(B卷)",
    "as32b": "s32b. 你滿不滿意和他們的即時通訊情形？(B卷)",
    "as32c": "s32c. 對於你和他們的即時通訊，符不符合以下情形？(B卷)",
    "as32c1": "s32c1. 增加彼此聯繫與訊息傳達。(B卷)",
    "as32c2": "s32c2. 增加彼此關心表達與心事分享。(B卷)",
    "as32c3": "s32c3. 減少彼此面對面互動。(B卷)",
    "as32c4": "s32c4. 引起彼此的衝突或誤解。(B卷)",
    "as33": "s33. 請問你和你兄弟姊妹間的即時通訊情形：(B卷)",
    "as33a": "s33a. 你平常和他們使用即時通訊的頻率如何？(B卷)",
    "as33b": "s33b. 你滿不滿意和他們的即時通訊情形？(B卷)",
    "as33c": "s33c. 對於你和他們的即時通訊，符不符合以下情形？(B卷)",
    "as33c1": "s33c1. 增加彼此聯繫與訊息傳達。(B卷)",
    "as33c2": "s33c2. 增加彼此關心表達與心事分享。(B卷)",
    "as33c3": "s33c3. 減少彼此面對面互動。(B卷)",
    "as33c4": "s33c4. 引起彼此的衝突或誤解。(B卷)",
    "as34": "s34. 在平日（上學日），請問你會不會上網？(A卷)",
    "as35": "s35. 在平日（上學日），請問你每天花多少時間上網做以下活動？(A卷)",
    "as35a": "s35a. 完成學校功課（查找完成作業需要的資料）(A卷)",
    "as35b": "s35b. 課外的學習（各種線上付費或免費的課程）(A卷)",
    "as35c": "s35c. 玩線上遊戲(A卷)",
    "as35d": "s35d. 看影片、聽音樂、迷因梗圖、卡通、漫畫(A卷)",
    "as35e": "s35e. 和他人聊天（傳訊息）(A卷)",
    "as35f": "s35f. 在網路上瀏覽自己有興趣的資訊(A卷)",
    "as36": "s36. 請問你使用網路搜尋過哪些健康議題？【可複選】",
    "as36_1": "s36. 皮膚健康", # 多選題拆分
    "as36_2": "s36. 身材體態",
    "as36_3": "s36. 營養",
    "as36_4": "s36. 性教育",
    "as36_5": "s36. 憂鬱",
    "as36_6": "s36. 焦慮",
    "as36_7": "s36. 壓力調適",
    "as36_8": "s36. 其他（請說明）",
    "aks36_8": "s36. 請問你使用網路搜尋過哪些健康議題？(10)其他（請說明）",
    "as36_9": "s36. 都沒有",
    "as37": "s37. 當你在現實生活遇到困難感到憂慮時，你會上網： (A卷)",
    "as37a": "s37a. 找人聊天給你支持或安慰。 (A卷)",
    "as37b": "s37b. 找人討論問題，設法解決困難。 (A卷)",
    "as37c": "s37c. 搜尋相關的資訊，設法解決困難。(A卷)",
    "as37d": "s37d. 瀏覽跟困難無關的資訊，轉移自己的注意力。(A卷)",
    "as37e": "s37e. 玩遊戲來減少困擾或壓力感受。(A卷)",
    "as37f": "s37f. 購物來改善心情或壓力感受。(A卷)",
    "as38": "s38. 以下描述符不符合你的個人經驗感受呢？(A卷)",
    "as38a": "s38a. 我喜歡自己在網路上的形象。(A卷)",
    "as38b": "s38b. 我在網路上可以表現出我最好的一面。(A卷)",
    "as38c": "s38c. 在網路世界中我覺得自己可以成為理想的自己。(A卷)",
    "as38d": "s38d. 我喜歡自己在現實生活裡的形象。(A卷)",
    "as38e": "s38e. 我在現實生活裡可以表現出我最好的一面。(A卷)",
    "as38f": "s38f. 在現實生活裡我覺得自己可以成為理想的自己。(A卷)",
    "as38g": "s38g. 我在網路世界裡經常會扮演成不一樣的角色。 (A卷)",
    "as38h": "s38h. 我在不同的社群媒體中（如臉書、IG）會有不一樣的形象。(A卷)",
    "as38i": "s38i. 我喜歡在網路上擁有多種不一樣的身分、形象或樣貌。(A卷)",
    "as38j": "s38j. 我在網路上呈現自我的方式，與在現實生活的自我並不相同。(A卷)",
    "as38k": "s38k. 我覺得我在網絡世界和現實世界中是不同的人。(A卷)",
    "as38l": "s38l. 我在網路世界中表現出與現實世界中截然不同的言行舉止。(A卷)",
    "as38m": "s38m. 比起在現實生活，我覺得自己在網路上比較輕鬆自在(A卷)",
    "as38n": "s38n. 我喜歡在網路上的生活，勝過於現實中的生活。(A卷)",
    "as38o": "s38o. 如果可以，我希望自己可以一直待在網路上。(A卷)",
    "as39": "s39. 你同不同意以下說法？(A卷)",
    "as39a": "s39a. 沒跟上朋友們在網路上聊天或開玩笑的話題，會讓我感到很焦慮。 (A卷)",
    "as39b": "s39b. 錯過和朋友們在網路上的聚會，會讓我感到很困擾。(A卷)",
    "as39c": "s39c. 我會隨時透過網路追蹤朋友們的行蹤和做的事情。 (A卷)",
    "as39d": "s39d. 沒跟上網路上的首播、直播或活動，我會感到很困擾。(A卷)",
    "as40": "s40. 關於你在網路上的意見表達，以下敘述你同不同意？(A卷)",
    "as40a": "s40a. 我經常會在網路上和他人比較自己在生活上獲得的成就。 (A卷)",
    "as40b": "s40b. 我總是在網路上留意他人的表現，並和自己的表現進行比較。(A卷)",
    "as40c": "s40c. 我經常在網路上和他人比較自己受歡迎的程度。(A卷)",
    "as40d": "s40d. 如果我想要更瞭解某些事情，我會試著去網路上看看別人的想法。(A卷)",
    "as40e": "s40e. 我總是好奇網路上的人們在和我遇到相同的問題時會怎麼想或怎麼做。(A卷)",
    "as40f": "s40f. 我喜歡在網路上看看其他人的不同觀點和經歷。(A卷)",
    "as41": "s41. 關於上網，以下敘述符不符合您的情形？(A卷)",
    "as41a": "s41a. 想上網而無法上網的時候，我就會感到坐立不安。(A卷)",
    "as41b": "s41b. 只要有一段時間沒有上網，我就會覺得心裡不舒服。(A卷)",
    "as41c": "s41c. 我只要超過一天沒上網玩，就會很難受。(A卷)",
    "as41d": "s41d. 發現自己上網休閒的時間越來越長。(A卷)",
    "as41e": "s41e. 平均而言，我每週上網休閒的時間比以前增加許多。(A卷)",
    "as41f": "s41f. 我實際上網所花的時間遠遠超過原來預期的時間。(A卷)",
    "as41g": "s41g. 上網對我的學業或工作已造成一些負面的影響。(A卷)",
    "as41h": "s41h. 上網已經很明顯地危害到我的生理或心理健康。(A卷)",
    "as41i": "s41i. 因為上網的關係，我和家人及朋友的互動減少了。(A卷)",
    "as41j": "s41j. 我常因熬夜上網休閒，導致白天精神或體力很差。(A卷)",
    # --- (四) 同儕關係與霸凌行為 ---
    # --- (五) 數位環境與素養 ---
    "as56a": "s56a. 請問你使用以下這些設備的頻率如何? 電腦(含桌機或筆電) (A卷)",

    # --- (六) 生活經驗與態度 ---
    "as74": "s74. 最近你的身高成長情形怎麼樣?",
    "as84": "s84. 請勾選出你認為最符合你目前體型的選項:",

    # --- 附加問卷相關 ---
    "attach": "附加問卷. 請問您是否有意願參與附加問卷填寫?",
    "attach2": "附加問卷. 填寫之附加問卷類型",
    "response": "附加問卷. 問卷作答類型",

}
# 您可以在需要時迭代這個字典或查詢特定變項的意義

In [19]:

# --- 如何使用這個 id_map ---
# 假設您已經手動補充完成上面的字典

# 獲取某個變項的說明
variable_id = "as1"
description = id_map.get(variable_id, f"{variable_id} 的說明未在 id_map 中定義")
print(f"變項 {variable_id} 的說明是: {description}")

variable_id = "as9a"
description = id_map.get(variable_id, f"{variable_id} 的說明未在 id_map 中定義")
print(f"變項 {variable_id} 的說明是: {description}")

variable_id = "as28" # 測試一個不存在的變項
description = id_map.get(variable_id, f"{variable_id} 的說明未在 id_map 中定義")
print(f"變項 {variable_id} 的說明是: {description}")


變項 as1 的說明是: s1. 請問你的性別(生理性別)?
變項 as9a 的說明是: s9a. 作決定時,家人會彼此商量。
變項 as28 的說明是: s28. 關於上網，你父親和你有沒有以下的情形？(B卷)


In [14]:
# 通用特殊值定義 (方便複製使用，但通常會直接寫入每個映射字典)
common_special_values = {
    -4: "不適用",
    -5: "不一定",
    -6: "跳答",
    -7: "不知道、不清楚、忘記了",
    -8: "拒答",
    -9: "系統遺漏值",
    -99: "無意義作答/邏輯矛盾",
    -999: "此卷未答"
}

# 範例 1: as4a1 (s4a(一). 請問你的第一位主要照顧者是你的?)
as4a1_value_map = {
    1: "父親",
    2: "母親",
    3: "繼父或養父",
    4: "繼母或養母",
    5: "祖父",
    6: "祖母",
    7: "其他", # 對應 (7) 其他，請說明：
    # 加入通用特殊值
    **common_special_values
}

# 範例 2: as1 (s1. 請問你的性別(生理性別)?)
as1_value_map = {
    1: "女",
    2: "男",
    # 加入通用特殊值
    **common_special_values
}

# 範例 3: as9a (s9a. 作決定時,家人會彼此商量。) - Likert Scale
as9a_value_map = {
    1: "很不符合",
    2: "不符合",
    3: "符合",
    4: "很符合",
    # 加入通用特殊值
    **common_special_values
}

# 範例 4: as5 (s5. 請問你是否有兄弟姐妹?) - 簡單是非題
as5_value_map = {
    1: "有",
    2: "沒有",
    # 加入通用特殊值
    **common_special_values
}

# 範例 5: as20 (s20. 你上學期的平均成績大約如何?) - 包含特定遺漏值
as20_value_map = {
    1: "全班五名以內",
    2: "全班六至十名",
    3: "全班十一至二十名",
    4: "全班二十一至三十名",
    5: "全班三十名以後",
    -7: "我不知道", # Codebook 特別定義了此題的 -7
    # 加入其他通用特殊值 (如果適用)
    -4: "不適用", -5: "不一定", -6: "跳答", -8: "拒答",
    -9: "系統遺漏值", -99: "無意義作答/邏輯矛盾", -999: "此卷未答"
}


# --- 步驟 2: 應用映射，創建帶有文字標籤的新欄位 ---
# 建議將轉換後的標籤存儲在新欄位中，以便保留原始數值欄位

# 使用 .map() 方法，它會自動將字典中未定義的 Key 映射為 NaN
# 檢查欄位是否存在是一個好習慣
if "as4a1" in df.columns:
    df["as4a1_label"] = df["as4a1"].map(as4a1_value_map)
else:
    print("警告：欄位 "as4a1" 不存在")

if "as1" in df.columns:
    df["as1_label"] = df["as1"].map(as1_value_map)
else:
    print("警告：欄位 "as1" 不存在")

if "as9a" in df.columns:
    df["as9a_label"] = df["as9a"].map(as9a_value_map)
else:
    print("警告：欄位 "as9a" 不存在")

if "as5" in df.columns:
    df["as5_label"] = df["as5"].map(as5_value_map)
else:
    print("警告：欄位 "as5" 不存在")

if "as20" in df.columns:
    df["as20_label"] = df["as20"].map(as20_value_map)
else:
    print("警告：欄位 "as20" 不存在")

# --- 步驟 3: 檢視結果 ---
# 選擇要顯示的欄位，只顯示存在的欄位
display_cols = []
for var in ["as1", "as5", "as9a", "as20", "as4a1"]:
    if var in df.columns:
        display_cols.extend([var, f"{var}_label"]) # 加入原始值和標籤欄位

if display_cols:
    print("\n轉換結果預覽 (部分欄位，前 5 行):")
    print(df[display_cols].head())
else:
    print("\n所有範例欄位都不在 DataFrame 中，無法顯示預覽。")


# --- 如何處理 "其他(請說明)" ---
# 對於像 as4a1 中數值 7 代表 "其他" 的情況，其對應的文字說明通常儲存在 aks4a1 欄位中。
# 您可以使用轉換後的 as4a1_label 來輔助查看 aks4a1 的內容：
# print(df.loc[df["as4a1_label"] == "其他", ["as4a1", "as4a1_label", "aks4a1"]].head())

警告：欄位 'as4a1' 不存在
警告：欄位 'as1' 不存在

轉換結果預覽 (部分欄位，前 5 行):
   as5 as5_label  as9a as9a_label  as20 as20_label
0    1         有     3         符合     2     全班六至十名
1    1         有     3         符合     2     全班六至十名
2    1         有     3         符合     1     全班五名以內
3    2        沒有     4        很符合     3   全班十一至二十名
4    1         有     4        很符合     3   全班十一至二十名
